# Get Target

In [ ]:
!pip install -r requirements_colab.txt -q

> To speed up the review process , i provided the ***drive id*** of the data i've created from the Train creation folder noteboooks .
---
> I  also add each data drive link in the Readme Pdf file attached with this solution
---
> The data used in this notebook is S2TrainObs1.csv

In [ ]:
!gdown --id 1hNRbtcqd9F6stMOK1xAZApDITwAjiSDJ

In [ ]:
#import necessary dependecies
import os
import warnings
import numpy as np  
import pandas as pd
import random
import gc
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

warnings.filterwarnings('ignore') 
np.random.seed(111)
random.seed(111)

In [ ]:
def create_target():
  train1 =pd.read_csv("Train_0_500.csv" )
  train2 =pd.read_csv("Train_500_1000.csv" )
  train3 =pd.read_csv("Train_1000_1500.csv" )
  train4 =pd.read_csv("Train_1500_2000.csv" )
  train5 =pd.read_csv("Train_2000_2250.csv" )
  train6 =pd.read_csv("Train_2250_End.csv" )
  

  train = pd.concat([train1,train2,train3,train4,train5,train6])
  train = train.groupby('field_id').median().reset_index().sort_values('field_id')
  train.label = train.label.astype('int')
  return train[['field_id','label']]

# Optimized Blend

In [ ]:
y_true = pd.get_dummies(create_target()['label']).values

In [ ]:
test_dict = {'CatboostS2': 'S2_Catboost.csv', 
             'LightGbmS2': 'S2_LightGBM.csv', 
             'XgboostS2' : 'S2_Xgboost.csv', 
             'NN_AttentionS2': 'S2_NNAttention.csv' ,
             'Neural_NetworkS2' : 'S2_NN.csv',
           }

BlendTest = np.zeros((len(test_dict), 35295,y_true.shape[1]))
for i in range(BlendTest.shape[0]):
    BlendTest[i] = pd.read_csv(list(test_dict.values())[i]).values[:,1:]

In [ ]:
BlendPreds = np.tensordot([0.25,0.325,0.175,0.125,0.125], BlendTest, axes = ((0), (0)))

In [ ]:
BlendPreds.shape

(35295, 9)

In [ ]:
pred_df = pd.DataFrame(BlendPreds)
pred_df = pred_df.rename(columns={
    0:'Crop_ID_1',
    1:'Crop_ID_2', 
    2:'Crop_ID_3',
    3:'Crop_ID_4',
    4:'Crop_ID_5',
    5:'Crop_ID_6',
    6:'Crop_ID_7',
    7:'Crop_ID_8',
    8:'Crop_ID_9'
})
pred_df['field_id'] = pd.read_csv("S1_10FoldsCatboost_V5.csv")['field_id'].astype('int').values
pred_df = pred_df[['field_id', 'Crop_ID_1', 'Crop_ID_2', 'Crop_ID_3', 'Crop_ID_4', 'Crop_ID_5', 'Crop_ID_6', 'Crop_ID_7', 'Crop_ID_8', 'Crop_ID_9']]
pred_df.head()

,field_id,Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7,Crop_ID_8,Crop_ID_9
0,30,0.045000,0.290610,0.008197,0.000064,0.211637,0.368469,7.276092e-02,2.197484e-03,0.001065
1,39,0.830454,0.097743,0.027974,0.023304,0.018698,0.001684,7.164873e-05,6.288274e-05,0.000009
2,49,0.053877,0.380285,0.001910,0.000864,0.001812,0.498334,5.676415e-02,4.029529e-03,0.002125
3,54,0.000003,0.000068,0.000065,0.999778,0.000080,0.000004,4.349412e-07,2.930337e-08,0.000002
4,56,0.782630,0.038284,0.001758,0.000415,0.001601,0.073110,9.841273e-02,3.738112e-03,0.000050


# Optimized Stacking

In [ ]:
y_true = create_target()

oof_dict = {'CatboostS2': 'S2_oof_cat.npy', 
            'LightGbm'  : 'S2_oof_lgbm.npy', 
            'XgboostS2' : 'S2_oof_xgb.npy', 
            'NN_AttentionS2'   : 'S2_oof_NNAttention.npy' ,
            'Neural_NetworkS2' : 'S2_NN.npy',}

oof_data = y_true.copy()
for i in range(len(list(oof_dict.values())) ):
    local = pd.DataFrame(np.load(list(oof_dict.values())[i]),columns=[f'{list(oof_dict.keys())[i]}_oof_{j}' for j in range(1,10)] )
    oof_data = pd.concat([oof_data,local],axis=1)

In [ ]:
oof_data.head()

,field_id,label,CatboostS2_oof_1,CatboostS2_oof_2,CatboostS2_oof_3,CatboostS2_oof_4,CatboostS2_oof_5,CatboostS2_oof_6,CatboostS2_oof_7,CatboostS2_oof_8,CatboostS2_oof_9,LightGbmS2_oof_1,LightGbmS2_oof_2,LightGbmS2_oof_3,LightGbmS2_oof_4,LightGbmS2_oof_5,LightGbmS2_oof_6,LightGbmS2_oof_7,LightGbmS2_oof_8,LightGbmS2_oof_9,XgboostS2_oof_1,XgboostS2_oof_2,XgboostS2_oof_3,XgboostS2_oof_4,XgboostS2_oof_5,XgboostS2_oof_6,XgboostS2_oof_7,XgboostS2_oof_8,XgboostS2_oof_9,NN_AttentionS2_oof_1,NN_AttentionS2_oof_2,NN_AttentionS2_oof_3,NN_AttentionS2_oof_4,NN_AttentionS2_oof_5,NN_AttentionS2_oof_6,NN_AttentionS2_oof_7,NN_AttentionS2_oof_8,NN_AttentionS2_oof_9,Neural_NetworkS2_oof_1,Neural_NetworkS2_oof_2,Neural_NetworkS2_oof_3,Neural_NetworkS2_oof_4,Neural_NetworkS2_oof_5,Neural_NetworkS2_oof_6,Neural_NetworkS2_oof_7,Neural_NetworkS2_oof_8,Neural_NetworkS2_oof_9
0,0.0,5,0.151103,0.408235,0.153073,0.057755,0.140192,0.015172,0.061491,0.001563,1.141486e-02,0.086760,0.467310,0.109283,0.045391,0.135258,0.024908,0.089605,0.000789,4.069682e-02,0.055040,0.528661,0.139965,0.026667,0.205064,0.015499,0.014682,0.000331,1.409152e-02,0.217523,0.254941,0.024358,0.000888,0.116820,0.069534,0.159333,8.371037e-03,1.482335e-01,0.248323,0.063602,0.165253,0.116913,0.085659,0.039060,0.071560,0.068990,1.406386e-01
1,1.0,4,0.000374,0.004996,0.001850,0.991610,0.000780,0.000336,0.000034,0.000019,3.108247e-08,0.000109,0.003216,0.000705,0.995636,0.000263,0.000055,0.000006,0.000009,3.054620e-07,0.000198,0.002835,0.000366,0.995879,0.000392,0.000285,0.000026,0.000019,2.579816e-07,0.000007,0.000595,0.000316,0.999062,0.000009,0.000009,0.000002,5.175813e-08,2.387890e-08,0.000004,0.000168,0.000199,0.999556,0.000006,0.000055,0.000010,0.000001,2.205914e-08
2,2.0,7,0.066400,0.236450,0.010485,0.002395,0.000971,0.194929,0.484263,0.003998,1.081458e-04,0.063340,0.117280,0.002518,0.000746,0.001697,0.142923,0.666481,0.004657,3.574596e-04,0.117672,0.088883,0.024491,0.004161,0.004631,0.282346,0.472852,0.002335,2.628644e-03,0.222095,0.062763,0.051674,0.000074,0.013310,0.392390,0.170866,7.996373e-02,6.863958e-03,0.041555,0.017205,0.007484,0.001634,0.000168,0.120018,0.711179,0.100295,4.616875e-04
3,3.0,6,0.004545,0.002535,0.000173,0.000169,0.000012,0.836919,0.154091,0.001555,5.051248e-07,0.002339,0.002473,0.000197,0.000478,0.000070,0.849142,0.144199,0.001095,6.850243e-06,0.001276,0.001899,0.000207,0.000249,0.000056,0.883793,0.111753,0.000762,3.862779e-06,0.009188,0.027352,0.000387,0.000663,0.000066,0.282435,0.679280,4.827284e-04,1.474093e-04,0.001675,0.009351,0.000388,0.000111,0.000209,0.322290,0.663514,0.002250,2.118325e-04
4,4.0,8,0.012316,0.285561,0.008110,0.007160,0.004621,0.096456,0.042729,0.543046,9.371442e-07,0.014424,0.397430,0.002354,0.007543,0.002032,0.092745,0.035764,0.447701,8.785577e-06,0.012603,0.745228,0.006414,0.004067,0.000781,0.088892,0.033576,0.108429,1.070976e-05,0.000366,0.000123,0.000020,0.000012,0.000007,0.002525,0.006274,9.906710e-01,1.403773e-06,0.000480,0.004208,0.000280,0.000170,0.001646,0.035514,0.110189,0.847494,1.798139e-05


In [ ]:
test_dict = {'CatboostS2': 'S2_Catboost.csv', 
             'LightGbmS2': 'S2_LightGBM.csv', 
             'XgboostS2' : 'S2_Xgboost.csv', 
             'NN_AttentionS2': 'S2_NNAttention.csv' ,
             'Neural_NetworkS2' : 'S2_NN.csv',
           }

test_data = pd.read_csv("S1_10FoldsCatboost_V5.csv")[['field_id']]
for i in range(len(list(test_dict.values()))):
    local = pd.DataFrame(pd.read_csv(list(test_dict.values())[i]).iloc[:,1:].values,columns=[f'{list(test_dict.keys())[i]}_oof_{j}' for j in range(1,10)] )
    test_data = pd.concat([test_data,local],axis=1)

In [ ]:
test_data.head()

,field_id,CatboostS2_oof_1,CatboostS2_oof_2,CatboostS2_oof_3,CatboostS2_oof_4,CatboostS2_oof_5,CatboostS2_oof_6,CatboostS2_oof_7,CatboostS2_oof_8,CatboostS2_oof_9,LightGbmS2_oof_1,LightGbmS2_oof_2,LightGbmS2_oof_3,LightGbmS2_oof_4,LightGbmS2_oof_5,LightGbmS2_oof_6,LightGbmS2_oof_7,LightGbmS2_oof_8,LightGbmS2_oof_9,XgboostS2_oof_1,XgboostS2_oof_2,XgboostS2_oof_3,XgboostS2_oof_4,XgboostS2_oof_5,XgboostS2_oof_6,XgboostS2_oof_7,XgboostS2_oof_8,XgboostS2_oof_9,NN_AttentionS2_oof_1,NN_AttentionS2_oof_2,NN_AttentionS2_oof_3,NN_AttentionS2_oof_4,NN_AttentionS2_oof_5,NN_AttentionS2_oof_6,NN_AttentionS2_oof_7,NN_AttentionS2_oof_8,NN_AttentionS2_oof_9,Neural_NetworkS2_oof_1,Neural_NetworkS2_oof_2,Neural_NetworkS2_oof_3,Neural_NetworkS2_oof_4,Neural_NetworkS2_oof_5,Neural_NetworkS2_oof_6,Neural_NetworkS2_oof_7,Neural_NetworkS2_oof_8,Neural_NetworkS2_oof_9
0,30,0.099415,0.191492,0.008146,0.000002,0.265454,2.954184e-01,1.377500e-01,1.703509e-03,6.185369e-04,0.026285,0.310600,0.008153,0.000051,0.170344,4.225246e-01,5.889915e-02,2.427002e-03,7.151681e-04,2.686587e-02,0.278273,0.005266,0.000036,0.226852,3.972440e-01,6.204018e-02,2.918740e-03,5.045718e-04,0.041120,0.309569,0.009097,0.000099,0.157301,0.451127,0.026184,2.467516e-03,0.003034,1.409206e-02,0.435187,0.011612,0.000229,0.244402,2.510796e-01,4.040882e-02,1.308897e-03,1.680860e-03
1,39,0.859933,0.077197,0.030461,0.018503,0.012923,9.373837e-04,1.386322e-05,3.270161e-05,1.548373e-08,0.861973,0.065584,0.026303,0.029184,0.015589,1.150208e-03,8.767029e-05,1.223347e-04,7.211912e-06,8.399638e-01,0.086238,0.041110,0.017728,0.014079,7.329827e-04,6.706321e-05,7.469813e-05,6.926843e-06,0.710284,0.205948,0.010874,0.023593,0.046379,0.002728,0.000153,7.473998e-06,0.000033,7.964015e-01,0.130348,0.026051,0.025137,0.017122,4.849683e-03,7.024565e-05,7.537185e-06,1.236286e-05
2,49,0.054238,0.406445,0.002309,0.000810,0.002537,4.616461e-01,6.756595e-02,3.637855e-03,8.122070e-04,0.071829,0.337148,0.002624,0.001245,0.001867,5.037783e-01,7.461354e-02,2.911400e-03,3.984989e-03,6.760062e-02,0.387984,0.002664,0.001403,0.001759,4.731688e-01,5.660396e-02,5.306844e-03,3.509160e-03,0.021883,0.448379,0.000042,0.000026,0.000337,0.498564,0.026004,4.699463e-03,0.000066,1.926275e-02,0.361248,0.000067,0.000067,0.001765,5.925541e-01,1.973689e-02,5.261837e-03,3.797240e-05
3,54,0.000006,0.000005,0.000056,0.999865,0.000066,7.809341e-07,2.470965e-08,4.544660e-09,7.148709e-07,0.000001,0.000003,0.000012,0.999971,0.000011,5.276792e-07,6.530829e-08,4.712327e-08,4.359477e-07,6.237326e-07,0.000001,0.000009,0.999980,0.000008,2.306626e-07,3.376910e-08,3.242743e-08,3.103384e-07,0.000009,0.000520,0.000347,0.998624,0.000455,0.000025,0.000003,5.715823e-08,0.000017,6.959021e-07,0.000005,0.000011,0.999970,0.000013,5.265201e-08,5.572116e-09,2.604579e-10,1.788761e-08
4,56,0.659646,0.061991,0.001754,0.000951,0.002519,1.109664e-01,1.573993e-01,4.767294e-03,6.040271e-06,0.803134,0.035978,0.001897,0.000286,0.000826,7.178973e-02,8.237881e-02,3.632498e-03,7.819724e-05,7.782459e-01,0.036528,0.001519,0.000284,0.001318,8.619568e-02,9.389127e-02,1.959667e-03,5.857029e-05,0.837220,0.029454,0.002752,0.000264,0.003474,0.031217,0.090314,5.212712e-03,0.000092,9.268380e-01,0.008156,0.000747,0.000014,0.000304,2.440462e-02,3.655615e-02,2.969571e-03,1.151106e-05


# Stacker-Model

In [ ]:

class AzerStacking :
  def __init__(self,y) :
    self.y = y
  
  def StackingRegressor(self ,KFOLD,stacking_train : pd.DataFrame , stacking_test : pd.DataFrame) :
      
      cols = stacking_train.drop(['field_id','label'],1).columns.tolist()
      X , y , Test = stacking_train[cols] , stacking_train['label'] , stacking_test[cols]
      final_preds = [] ; err_cb = []
      oof_stack = np.zeros((len(X),9)) ;
      
      for fold,(train_index, test_index) in enumerate(KFOLD.split(X,y)):
          X_train, X_test = X.values[train_index], X.values[test_index]
          y_train, y_test = y.values[train_index], y.values[test_index]

          model1 = LGBMClassifier(verbose=10)
          model2 = CatBoostClassifier(iterations=50,verbose=0)
          
          model1.fit(X_train,y_train)
          model2.fit(X_train,y_train)
          
          preds1=model1.predict_proba(X_test)
          preds2=model2.predict_proba(X_test)
          
          preds = preds1*0.7+preds2*0.3
          oof_stack[test_index] = preds
          err_cb.append(log_loss(y_test,preds))
          print(f'logloss fold-{fold+1}/10',log_loss(y_test,preds))

          test_pred1 = model1.predict_proba(Test.values)
          test_pred2 = model2.predict_proba(Test.values)
          test_pred = test_pred1*0.7+test_pred2*0.3
          final_preds.append(test_pred)
      
      print(2*'--------------------------------------')
      print('STACKING Log Loss',log_loss(y, oof_stack))

      return oof_stack,np.mean(final_preds,axis=0)

In [ ]:
folds = StratifiedKFold(n_splits=10,shuffle=True,random_state=47) 
AzerStacker = AzerStacking(y=oof_data['label'])

In [ ]:
oof_stack,stack_preds  = AzerStacker.StackingRegressor(KFOLD=folds ,stacking_train=oof_data ,stacking_test=test_data)

logloss fold-1/10 0.5575429309132016
logloss fold-2/10 0.5480782111642241
logloss fold-3/10 0.5388301979374165
logloss fold-4/10 0.5637427543501794
logloss fold-5/10 0.5507966995760387
logloss fold-6/10 0.5481787880475464
logloss fold-7/10 0.5613963657697073
logloss fold-8/10 0.5460696611749111
logloss fold-9/10 0.5457205682181677
logloss fold-10/10 0.5618579369661109
----------------------------------------------------------------------------
STACKING Log Loss 0.5522214034732872


In [ ]:
pred_df = pd.DataFrame(stack_preds)
pred_df = pred_df.rename(columns={
    0:'Crop_ID_1',
    1:'Crop_ID_2', 
    2:'Crop_ID_3',
    3:'Crop_ID_4',
    4:'Crop_ID_5',
    5:'Crop_ID_6',
    6:'Crop_ID_7',
    7:'Crop_ID_8',
    8:'Crop_ID_9'
})
pred_df['field_id'] = pd.read_csv("S1_10FoldsCatboost_V5.csv")['field_id'].astype('int').values
pred_df = pred_df[['field_id', 'Crop_ID_1', 'Crop_ID_2', 'Crop_ID_3', 'Crop_ID_4', 'Crop_ID_5', 'Crop_ID_6', 'Crop_ID_7', 'Crop_ID_8', 'Crop_ID_9']]
pred_df.head()

,field_id,Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7,Crop_ID_8,Crop_ID_9
0,30,0.035901,0.241092,0.009310,0.001260,0.316071,0.312686,0.078818,0.003278,0.001585
1,39,0.842662,0.059972,0.039873,0.027730,0.026674,0.001937,0.000759,0.000239,0.000153
2,49,0.051153,0.308816,0.003595,0.001922,0.002799,0.552970,0.070317,0.007154,0.001274
3,54,0.000335,0.001100,0.001665,0.992045,0.000715,0.000268,0.003818,0.000022,0.000031
4,56,0.749472,0.047506,0.003979,0.001515,0.002559,0.072030,0.114923,0.007775,0.000240


# Stacking - Blend 

In [ ]:
pred_df = pd.DataFrame(stack_preds*0.65+ BlendPreds*0.35)
pred_df = pred_df.rename(columns={
    0:'Crop_ID_1',
    1:'Crop_ID_2', 
    2:'Crop_ID_3',
    3:'Crop_ID_4',
    4:'Crop_ID_5',
    5:'Crop_ID_6',
    6:'Crop_ID_7',
    7:'Crop_ID_8',
    8:'Crop_ID_9'
})
pred_df['field_id'] = pd.read_csv("S1_10FoldsCatboost_V5.csv")['field_id'].astype('int').values
pred_df = pred_df[['field_id', 'Crop_ID_1', 'Crop_ID_2', 'Crop_ID_3', 'Crop_ID_4', 'Crop_ID_5', 'Crop_ID_6', 'Crop_ID_7', 'Crop_ID_8', 'Crop_ID_9']]
pred_df.head()

,field_id,Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7,Crop_ID_8,Crop_ID_9
0,30,0.039085,0.258423,0.008921,0.000841,0.279519,0.332210,0.076698,0.002900,0.001403
1,39,0.838389,0.073191,0.035709,0.026181,0.023883,0.001849,0.000519,0.000177,0.000103
2,49,0.052107,0.333830,0.003005,0.001552,0.002453,0.533847,0.065573,0.006060,0.001572
3,54,0.000219,0.000739,0.001105,0.994752,0.000493,0.000176,0.002482,0.000014,0.000021
4,56,0.761078,0.044278,0.003202,0.001130,0.002224,0.072408,0.109145,0.006362,0.000174


In [ ]:
# Write the predicted probabilites to a csv for submission
pred_df.to_csv('S2_Stacking_Blending.csv', index=False)